<a href="https://colab.research.google.com/github/GGMmattos/PIC/blob/main/Experimento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importação das bibliotecas

In [ ]:
#Importing needed libraries
!pip install -U scikit-learn

from pandas.core.window.ewm import OnlineExponentialMovingWindow
import pandas as pd #Biblioteca para carregamento de arquivo csv
import numpy as np
import seaborn as sns #Para visualização de gráficos
import matplotlib.pyplot as plt #Para visualização de gráficos
import plotly.express as px #Biblioteca para geração de gráfico dinámico
%matplotlib inline
from collections import Counter


#NLTK tools for text processing

import re, nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()


#Modeling packages

from sklearn.model_selection import train_test_split
from sklearn.metrics._plot.confusion_matrix import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity ='all'


!pip install -U spacy
!python -m spacy download pt
import spacy



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 102.8 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.5.2
    Uninstalling spacy-3.5.2:
      Successfully uninstalled spacy-3.5.2
2023-06-22 02:58:29.808466: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-22 02:58:30.716586: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
⚠ As of spaCy v3.0, shortcuts like 'pt' are deprecated. Please use the
full pipeline package name 'pt_core_news_sm' instead.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).



# RODADA 1


**Dataset Hate BR**

Sem pré-processamento

In [ ]:
from pandas.core.groupby import base
base_hatebr = pd.read_csv('/content/drive/MyDrive/PIC /Dataset/HateBR.csv')

In [ ]:
base_hatebr.offensive_language.value_counts()

1    3500
0    3500
Name: offensive_language, dtype: int64

Dividindo em base de teste e base de treinamento



In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    base_hatebr.instagram_comments,
    base_hatebr.offensive_language,
    test_size = 0.2, #20% das amostras vão para o conjunto de dados de teste
    random_state = 2022,
    stratify = base_hatebr.offensive_language
)

In [ ]:
print('Shape of X_train', X_train.shape)
print('Shape of X_test', X_test.shape )

Shape of X_train (5600,)
Shape of X_test (1400,)


In [ ]:
y_train.value_counts()

1    2800
0    2800
Name: offensive_language, dtype: int64

In [ ]:
y_test.value_counts()

1    700
0    700
Name: offensive_language, dtype: int64

Treinamento com Naive Bayes




In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

#Criação do objeto Pipeline
clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()), #using the ngram_range parameter
    ('Multi NB', MultinomialNB())
])

clf.fit(X_train, y_train)

#Obtem a precisão de X_test e e armazena em y_pred
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('Multi NB', MultinomialNB())])

              precision    recall  f1-score   support

           0       0.85      0.81      0.83       700
           1       0.82      0.86      0.84       700

    accuracy                           0.84      1400
   macro avg       0.84      0.84      0.84      1400
weighted avg       0.84      0.84      0.84      1400



Support Vector Machine(SVM)

In [ ]:
from sklearn.svm import SVC

clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()),
    ('SVM', SVC( kernel = 'linear'))
])

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('SVM', SVC(kernel='linear'))])

              precision    recall  f1-score   support

           0       0.84      0.85      0.84       700
           1       0.85      0.84      0.84       700

    accuracy                           0.84      1400
   macro avg       0.84      0.84      0.84      1400
weighted avg       0.84      0.84      0.84      1400



Multilayer Perceptron(MLP)

In [ ]:
from sklearn.neural_network import MLPClassifier

clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()),
    ('MLP', MLPClassifier())
])

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('MLP', MLPClassifier())])

              precision    recall  f1-score   support

           0       0.83      0.85      0.84       700
           1       0.85      0.83      0.84       700

    accuracy                           0.84      1400
   macro avg       0.84      0.84      0.84      1400
weighted avg       0.84      0.84      0.84      1400



 Regressão Logística(LR)

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()),
    ('LR', LogisticRegression())
])

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('LR', LogisticRegression())])

              precision    recall  f1-score   support

           0       0.83      0.82      0.83       700
           1       0.82      0.83      0.83       700

    accuracy                           0.83      1400
   macro avg       0.83      0.83      0.83      1400
weighted avg       0.83      0.83      0.83      1400



Com pre-processamento



In [ ]:
# comments = []
# stop_words = set(stopwords.words('portuguese')) #setando o banco de stopwords em PT-BR

# for words in base_hatebr ['instagram_comments']:
#   only_latters =re.sub("[^a-zA-Z]", " ", words) # somente letras
#   tokens = nltk.word_tokenize(only_latters) # tokenize the sentence
#   lower_case = [l.lower() for l in tokens] # converter all letters to low case
#   filtered_result = list(filter(lambda l: l not in stop_words, lower_case)) # Remove stopwords from the comments
#   comments.append(' '.join(filtered_result))

  # for i in range (len(comments)):
  #   words = nltk.word_tokenize(comments[i])
  #   newwords = [lemmatizer.lemmatize(word) for word in words]
  #   comments[i] = ' '.join(newwords)

Função de pré-processamento



*   Foi notado que ao realizar o procedimento de deixar somente letras o modelo teve piora de pontuação F1, sem a função o texto continua com emojis e caracteres especiais...



In [ ]:
import spacy
npl = spacy.load("pt_core_news_sm")
def preprocess(text):
  comentarios = text.lower()
  # only_latters =re.sub("[^a-zA-Z]", " ", comentarios) # somente letras
  doc = npl(comentarios)
  filtered_result = []
  for token in doc:
    if token.is_stop or token.is_punct :
      continue
    filtered_result.append(token.lemma_)
  return " ".join(filtered_result)

Criando uma nova coluna com o texto com pré-processamento

In [ ]:
# print(preprocess(base_hatebr['instagram_comments']))
base_hatebr['preprocessed_comments'] =  base_hatebr['instagram_comments'].apply(preprocess)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    base_hatebr.	preprocessed_comments,
    base_hatebr.offensive_language,
    test_size = 0.2, #20% das amostras vão para o conjunto de dados de teste
    random_state = 2022,
    stratify = base_hatebr.offensive_language
)

In [ ]:
print('Shape of X_train', X_train.shape)
print('Shape of X_test', X_test.shape )

Shape of X_train (5600,)
Shape of X_test (1400,)


Treinamento com Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report


#Criação do objeto Pipeline
clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()), #using the ngram_range parameter
    ('Multi NB', MultinomialNB())
])

clf.fit(X_train, y_train)

#Obtem a precisão de X_test e e armazena em y_pred
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('Multi NB', MultinomialNB())])

              precision    recall  f1-score   support

           0       0.86      0.84      0.85       700
           1       0.84      0.86      0.85       700

    accuracy                           0.85      1400
   macro avg       0.85      0.85      0.85      1400
weighted avg       0.85      0.85      0.85      1400



In [ ]:

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report #comparação entre os resultados
accuracy_score(y_test, y_pred) #retorna a porcentagem dos acertos


0.8492857142857143

Support Vector Machine(SVM)

In [ ]:
from sklearn.svm import SVC

clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()),
    ('SVM', SVC(C=2.0, kernel = 'linear'))
])

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('SVM', SVC(C=2.0, kernel='linear'))])

              precision    recall  f1-score   support

           0       0.84      0.88      0.86       700
           1       0.87      0.83      0.85       700

    accuracy                           0.85      1400
   macro avg       0.85      0.85      0.85      1400
weighted avg       0.85      0.85      0.85      1400



Multilayer Perceptron(MLP)

In [ ]:
from sklearn.neural_network import MLPClassifier

clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()),
    ('MLP', MLPClassifier())
])

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('MLP', MLPClassifier())])

              precision    recall  f1-score   support

           0       0.82      0.88      0.85       700
           1       0.87      0.81      0.84       700

    accuracy                           0.84      1400
   macro avg       0.84      0.84      0.84      1400
weighted avg       0.84      0.84      0.84      1400



 Regressão Logística(LR)

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()),
    ('LR', LogisticRegression())
])

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('LR', LogisticRegression())])

              precision    recall  f1-score   support

           0       0.84      0.87      0.85       700
           1       0.87      0.83      0.85       700

    accuracy                           0.85      1400
   macro avg       0.85      0.85      0.85      1400
weighted avg       0.85      0.85      0.85      1400



**Experimento com a TOLD BR**

Sem pré-processamento

In [ ]:
told_br = pd.read_csv("/content/drive/MyDrive/PIC /Dataset/ToLD-BR_alpha.csv")


Tranformação em base binária

In [ ]:
told_br["toxic"] = 0
told_br["toxic_1"] = 0
told_br["toxic_2"] = 0
told_br["toxic_3"] = 0
told_br["homophobia"] = 0
told_br["obscene"] = 0
told_br["insult"] = 0
told_br["racism"] = 0
told_br["misogyny"] = 0
told_br["xenophobia"] = 0
for i, row in told_br.iterrows():
    count = 0
    if 1 in list(row[["homophobia_1", "obscene_1", "insult_1", "racism_1", "misogyny_1", "xenophobia_1"]]):
        told_br.loc[i, "toxic_1"] = 1
        count += 1
    if 1 in list(row[["homophobia_2", "obscene_2", "insult_2", "racism_2", "misogyny_2", "xenophobia_2"]]):
        told_br.loc[i, "toxic_2"] = 1
        count += 1
    if 1 in list(row[["homophobia_3", "obscene_3", "insult_3", "racism_3", "misogyny_3", "xenophobia_3"]]):
        told_br.loc[i, "toxic_3"] = 1
        count += 1

    told_br.loc[i, "homophobia"] = np.sum(list(row[["homophobia_1", "homophobia_2", "homophobia_3"]]))

    told_br.loc[i, "obscene"] = np.sum(list(row[["obscene_1", "obscene_2", "obscene_3"]]))

    told_br.loc[i, "insult"] = np.sum(list(row[["insult_1", "insult_2", "insult_3"]]))

    told_br.loc[i, "racism"] = np.sum(list(row[["racism_1", "racism_2", "racism_3"]]))

    told_br.loc[i, "misogyny"] = np.sum(list(row[["misogyny_1", "misogyny_2", "misogyny_3"]]))

    told_br.loc[i, "xenophobia"] = np.sum(list(row[["xenophobia_1", "xenophobia_2", "xenophobia_3"]]))

    if count >= 1:
        told_br.loc[i, "toxic"] = 1

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    told_br.text,
    told_br.toxic,
    test_size = 0.2, #20% das amostras vão para o conjunto de dados de teste
    random_state = 2022,
    stratify = told_br.toxic
)

In [ ]:
print('Shape of X_train', X_train.shape)
print('Shape of X_test', X_test.shape )

Shape of X_train (16800,)
Shape of X_test (4200,)


Treinamento com Naive Bayes


In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

#Criação do objeto Pipeline
clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()), #using the ngram_range parameter
    ('Multi NB', MultinomialNB())
])

clf.fit(X_train, y_train)

#Obtem a precisão de X_test e e armazena em y_pred
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('Multi NB', MultinomialNB())])

              precision    recall  f1-score   support

           0       0.68      0.87      0.76      2349
           1       0.74      0.47      0.58      1851

    accuracy                           0.69      4200
   macro avg       0.71      0.67      0.67      4200
weighted avg       0.70      0.69      0.68      4200



Support Vector Machine(SVM)

In [ ]:
from sklearn.svm import SVC

clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()),
    ('SVM', SVC( kernel = 'linear'))
])

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('SVM', SVC(kernel='linear'))])

              precision    recall  f1-score   support

           0       0.77      0.80      0.78      2349
           1       0.73      0.69      0.71      1851

    accuracy                           0.75      4200
   macro avg       0.75      0.75      0.75      4200
weighted avg       0.75      0.75      0.75      4200



Multilayer Perceptron(MLP)

In [ ]:
from sklearn.neural_network import MLPClassifier

clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()),
    ('MLP', MLPClassifier())
])

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('MLP', MLPClassifier())])

              precision    recall  f1-score   support

           0       0.72      0.74      0.73      2349
           1       0.66      0.63      0.65      1851

    accuracy                           0.69      4200
   macro avg       0.69      0.69      0.69      4200
weighted avg       0.69      0.69      0.69      4200



 Regressão Logística(LR)

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()),
    ('LR', LogisticRegression())
])

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('LR', LogisticRegression())])

              precision    recall  f1-score   support

           0       0.75      0.83      0.79      2349
           1       0.75      0.65      0.69      1851

    accuracy                           0.75      4200
   macro avg       0.75      0.74      0.74      4200
weighted avg       0.75      0.75      0.75      4200



In [ ]:
#Comando para baixar o Dataset
told_br.to_csv("told_br_binary.csv")

Com pré-processamento

In [ ]:
#ja foi criada acima, excluir após testes
npl = spacy.load("pt_core_news_sm")
def preprocess(text):
  comentarios = text.lower()
  # only_latters =re.sub("[^a-zA-Z]", " ", comentarios) # somente letras
  doc = npl(comentarios)
  filtered_result = []
  for token in doc:
    if token.is_stop or token.is_punct :
      continue
    filtered_result.append(token.lemma_)
  return " ".join(filtered_result)

In [ ]:
# print(preprocess(base_hatebr['instagram_comments']))
told_br['preprocessed_text'] =  told_br['text'].apply(preprocess)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    told_br.preprocessed_text,
    told_br.toxic,
    test_size = 0.2, #20% das amostras vão para o conjunto de dados de teste
    random_state = 2022,
    stratify = told_br.toxic
)

Treinamento com Naive Bayes


In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()),
    ('Multi NB', MultinomialNB())
])

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('Multi NB', MultinomialNB())])

              precision    recall  f1-score   support

           0       0.70      0.84      0.76      2349
           1       0.73      0.53      0.61      1851

    accuracy                           0.71      4200
   macro avg       0.71      0.69      0.69      4200
weighted avg       0.71      0.71      0.70      4200



Support Vector Machine(SVM)

In [ ]:
from sklearn.svm import SVC

clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()),
    ('SVM', SVC( kernel = 'linear'))
])

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('SVM', SVC(kernel='linear'))])

              precision    recall  f1-score   support

           0       0.75      0.80      0.77      2349
           1       0.72      0.67      0.69      1851

    accuracy                           0.74      4200
   macro avg       0.74      0.73      0.73      4200
weighted avg       0.74      0.74      0.74      4200



Multilayer Perceptron(MLP)

In [ ]:
from sklearn.neural_network import MLPClassifier

clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()),
    ('MLP', MLPClassifier())
])

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('MLP', MLPClassifier())])

              precision    recall  f1-score   support

           0       0.73      0.72      0.72      2349
           1       0.65      0.66      0.65      1851

    accuracy                           0.69      4200
   macro avg       0.69      0.69      0.69      4200
weighted avg       0.69      0.69      0.69      4200



 Regressão Logística(LR)

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()),
    ('LR', LogisticRegression())
])

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('LR', LogisticRegression())])

              precision    recall  f1-score   support

           0       0.74      0.84      0.79      2349
           1       0.75      0.63      0.68      1851

    accuracy                           0.74      4200
   macro avg       0.75      0.73      0.74      4200
weighted avg       0.75      0.74      0.74      4200



# RODADA 2

Algumas importação para realizar o treinamento

In [ ]:
!pip install wandb
!pip install --upgrade simpletransformers
!pip install --upgrade transformers


from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging
import numpy as np
import sklearn
import pickle

from transformers import AutoTokenizer  # Or BertTokenizer
from transformers import AutoModelForPreTraining  # Or BertForPreTraining for loading pretraining heads
from transformers import AutoModel  # or BertModel, for BERT without pretraining heads
from transformers import pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


BERT IMBAU - Sem pré-processamento

In [ ]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)


told_br = pd.read_csv("/content/drive/MyDrive/PIC /Dataset/told_br_binary.csv")

# Preparing train data
Told_br_train_data = told_br.iloc[:,[1, 24]]
train,eva = train_test_split(Told_br_train_data,test_size = 0.2)

train_df = pd.DataFrame({
    'text': train['text'],
    'label': train['toxic_1']
})

eval_df = pd.DataFrame({
    'text': eva['text'],
    'label': eva['toxic_1']
})

#Optional model configuration
model_args = ClassificationArgs(num_train_epochs=1)

#Create a ClassificationModel
model = ClassificationModel(
    "bert", "neuralmind/bert-base-portuguese-cased", args=model_args,
    use_cuda=False
)

model.train_model(train_df)


Verificando accuracy

In [ ]:
lst = []
for arr in model_outputs:
    lst.append(np.argmax(arr))
true = eval_df['label'].tolist()
predicted = lst
sklearn.metrics.accuracy_score(true,predicted)

In [ ]:
# salvar modelo
import pickle
# salvar o modelo XGBoost (xgb_model) no arquivo sale_xgboost.pkl
with open('BERT-PR', 'wb') as file:
    pickle.dump(model, file)



BERT IMBAU - com pré-processamento

In [ ]:
told_br = pd.read_csv("/content/drive/MyDrive/PIC /Dataset/told_br_binary.csv")

In [ ]:
npl = spacy.load("pt_core_news_sm")
def preprocess(text):
  comentarios = text.lower()
  # only_latters =re.sub("[^a-zA-Z]", " ", comentarios) # somente letras
  doc = npl(comentarios)
  filtered_result = []
  for token in doc:
    if token.is_stop or token.is_punct :
      continue
    filtered_result.append(token.lemma_)
  return " ".join(filtered_result)

# print(preprocess(base_hatebr['instagram_comments']))
told_br['preprocessed_comments'] =  told_br['text'].apply(preprocess)

In [ ]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Preparing train data
Told_br_train_data = told_br.iloc[:,[33,24]]
train,eva = train_test_split(Told_br_train_data,test_size = 0.2)

train_df = pd.DataFrame({
    'text': train['preprocessed_comments'],
    'label': train['toxic_1']
})

eval_df = pd.DataFrame({
    'text': eva['preprocessed_comments'],
    'label': eva['toxic_1']
})

#Optional model configuration
model_args = ClassificationArgs(num_train_epochs=1)

#Create a ClassificationModel
model = ClassificationModel(
    "bert", "neuralmind/bert-base-portuguese-cased", args=model_args,
    use_cuda=False
)

model.train_model(train_df)

result, model_outputs, wrong_predictions = model.eval_model(eval_df)


Verificando accuracy

In [ ]:
lst = []
for arr in model_outputs:
    lst.append(np.argmax(arr))
true = eval_df['label'].tolist()
predicted = lst
sklearn.metrics.accuracy_score(true,predicted)


0.7754761904761904

In [ ]:

# salvar o modelo
with open('BERT-prep.pk', 'wb') as file:
    pickle.dump(model, file)


https://simpletransformers.ai/docs/tips-and-tricks/#additional-evaluation-metrics

In [ ]:
from simpletransformers.classification import ClassificationModel
import sklearn


model = ClassficationModel("bert", "bert-base-cased")

model.train_model(train_df, acc=sklearn.metrics.accuracy_score)

model.eval_model(eval_df, acc=sklearn.metrics.accuracy_score)

AUTO ML


In [ ]:
# !sudo apt-get install build-essential swig
!pip3 install --upgrade pip
!pip3 install auto-sklearn
!pip3 install liac-arff
import autosklearn.classification


In [ ]:
# cls = autosklearn.classification.AutoSklearnClassifier(
#     time_left_for_this_task=600,
#     max_models_on_disc=5,
#     memory_limit = 102400,
#     ensemble_size = 3)

# bow = CountVectorizer()
# bow.fit(X_train)

# X_TRAIN = bow.transform(X_train)
# y_TRAIN = list(y_train)

# cls.fit(X_TRAIN, y_TRAIN)
# predictions = cls.predict(X_test)